In [ ]:
import os
import sys
import xarray as xr
import cftime
import numpy as np
import datetime
import time as t_util


## Define experiment

In [ ]:
experiment = 'S3'

In [ ]:
dir_trendy    = '/Trendy/Data/'
dir_data_orig = '/Trendy/Data/Trendy-v10_' + experiment + '/'
dir_data_corr = '/Trendy/Data/Trendy-v10_' + experiment + '_corr/'
dir_countries = '/Trendy/Data/data_ancillary/info_countries/'
dir_forest    = '/GCB2021_commentary/Data/forest_masks/'
dir_LSMs      = '/Trendy/Data/LSMs_Trendy-v10/LSMs_unified/'
dir_LSMs_out  = '/Trendy/Data/LSMs_Trendy-v10/'
dir_tmp       = '/Trendy/Data/tmp/'
dir_grids     = '/Trendy/Data/grids/'
dir_out       = '/Trendy/Data/SLAND_Trendy-v10_' + experiment + '_LatLon/'
if not os.path.exists(dir_out):  os.mkdir(dir_out)
    

## Create reference grid

In [ ]:
#Select reference grid
sel_ref = 'CountryMask'
# sel_ref = 'LPJ-GUESS'
sel_ref = 'ForestMask'

#Select input files
if sel_ref=='CountryMask':
    fname_in = dir_countries + 'wrld_cntrs_BLUE_TN_upd.nc'
    variab   = 'ISOcode'
    annex    = ''
elif sel_ref=='ForestMask':
    fname_in = dir_forest + 'Hansen2010_IFL_2013.nc'
    variab   = 'Band1'
    annex    = '-ForestMask'
elif sel_ref=='LPJ-GUESS':
    fname_in = dir_data_orig + 'LPJ-GUESS/LPJ-GUESS_S2_nbp.nc'
    variab   = 'nbp'
    annex    = ''    

#Get size of lat and lon
dat_ref = xr.open_dataset(fname_in)
if 'latitude' in dat_ref.dims:  lat_name, lon_name = 'latitude', 'longitude'
else:                           lat_name, lon_name = 'lat', 'lon'    

#File name for grid description
grid_size = str(len(dat_ref[lat_name])) + 'x' + str(len(dat_ref[lon_name]))
fname_grid = dir_grids + 'grid_xy_' + grid_size + annex

#Create reference grid
if os.path.exists(fname_grid): os.remove(fname_grid)
os.system('cdo -s griddes -selvar,' + variab + ' ' + fname_in + ' > ' + fname_grid)


## Prepare land-sea mask for plotting maps (remap to fit correct grid)

In [ ]:
#Define grid resolution for remapping
resolution = '360x720'#'720x1440'#'360x720-ForestMask'#

#Define file names
file_grid    = dir_grids + 'grid_xy_' + resolution
fname_LSM    = dir_LSMs + 'LandFrac_DLEM.nc'
fname_remap  = dir_LSMs + 'LandFrac_DLEM_remap.nc'
fname_LSMout = dir_LSMs_out + 'LandSeaMask_' + resolution + '.nc'
if os.path.exists(fname_remap):   os.remove(fname_remap)
if os.path.exists(fname_LSMout):  os.remove(fname_LSMout)
    
#Remap
os.system('cdo remapbil,' + file_grid + ' ' + fname_LSM + ' ' + fname_remap)

#Open and convert to land-sea mak
data_LSM = xr.open_dataset(fname_remap)
data_LSM = 1 * (data_LSM.LandFrac>0)
data_LSM = data_LSM.to_dataset(name='sftlf')

#Save in NetCDF
data_LSM.to_netcdf(fname_LSMout)

#Delete temporary file
os.remove(fname_remap)


## Calculate yearly SLAND from NBP

In [ ]:
models = [#'CABLE-POP', 'CLASSIC', 'CLASSIC-N', 'CLM5.0', 'DLEM', 'IBIS', 'ISAM', 
          'ISBA-CTRIP', 
          'JSBACH', 'JULES-ES-1.1', 'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEE', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']

resolutions = ['360x720', '720x1440']#, '360x720-ForestMask']

#Define compression level
comp = dict(zlib=True, complevel=2)

#Loop over models
for model in models:
    
    print(model)
    
    #Define folder and get file name
    dir_model_orig = dir_data_orig + model + '/'
    dir_model_corr = dir_data_corr + model + '/'
    files_nbp_orig = [dir_model_orig + file for file in os.listdir(dir_model_orig) if '_nbp' in file and 'nbppft' not in file and experiment + '_' in file]
    
    #Check if corrected file exists
    if os.path.exists(dir_model_corr):
        files_nbp_corr = [dir_model_corr + file for file in os.listdir(dir_model_corr) if '_nbp' in file and 'nbppft' not in file and experiment + '_' in file]
        N = len(files_nbp_corr)
    else:
        N = 0
        
    #Select file name
    if N>0:  files_nbp = files_nbp_corr
    else:    files_nbp = files_nbp_orig
        
    #Check if file is unique
    if len(files_nbp)!=1:
        sys.exit('Filename not unique')
    else:
        file_nbp = files_nbp[0]
    
    #Define variable name for time
    if model in ['ISBA-CTRIP', 'ORCHIDEEv3']:
        time_var = 'time_counter'
    else:
        time_var = 'time'

    #Read data and check if it is monthly or yearly
    data_nbp = xr.open_dataset(file_nbp, use_cftime=True)
    if np.mean(np.diff(data_nbp[time_var]))>datetime.timedelta(days=180):
        sel = 'year'
    else:
        sel = 'month'        
    
    #Define file names
    fname_Ysum_tmp  = dir_tmp + model + '_NBP_latlon_yearsum_tmp.nc'
    fname_Ysum      = dir_tmp + model + '_NBP_latlon_yearsum.nc'
    fname_area      = dir_tmp + model + '_NBP_latlon_gridarea.nc'
    fname_area_reg  = dir_tmp + model + '_NBP_latlon_gridarea_regrid.nc'
    fname_flux      = dir_tmp + model + '_NBP_latlon_flux_tmp.nc'
    fname_SLden     = dir_tmp + model + '_' + experiment + '_SLANDdensity_NBP.nc'  
    fname_SLAND     = dir_out + model + '_' + experiment + '_SLAND_NBP.nc'  
    if os.path.exists(fname_SLAND):      os.remove(fname_SLAND)
        
    #Calculate yearly values and save in new file
    if model=='VISIT':
        
        #Open dataset
        data_VISIT = xr.open_dataset(file_nbp)

        #Calculate yearly values and remove temporary file
        if sel=='year':
            N_days = 365 + data_VISIT.time.dt.is_leap_year
            data_VISIT = data_VISIT * 86400 * N_days
        elif sel=='month':
            N_days = data_VISIT.time.dt.days_in_month
            data_VISIT = data_VISIT * 86400 * N_days
        
        data_VISIT = data_VISIT.resample(time='1Y').sum('time')
        data_VISIT.to_netcdf(fname_Ysum_tmp)

    else:
        if sel=='year':
            os.system("cdo -muldpy -mulc,86400 " + file_nbp + " " + fname_Ysum_tmp)
        elif sel=='month':
            os.system("cdo -yearsum -muldpm -mulc,86400 " + file_nbp + " " + fname_Ysum_tmp)
    
    #Read data
    data = xr.open_dataset(fname_Ysum_tmp, engine='netcdf4')

    #Rename variable for SDGVM
    if 'nbp_annual' in data.data_vars:  data = data.rename({'nbp_annual': 'nbp'})
        
    #Remove data variables that are not necessary
    vars_del = [var for var in data.data_vars if var!='nbp']
    data = data.drop(vars_del)
    data = data.fillna(0)
    data.to_netcdf(fname_Ysum)
    
    #Get grid area
    os.system('cdo gridarea ' + fname_Ysum + ' ' + fname_area)
        
    #Multiply land fraction for models for which it is necessary
    files_LSM = [file for file in os.listdir(dir_LSMs) if (model + '.nc' in file) and ('CellArea' not in file)]
    if model in ['CLM5.0', 'DLEM', 'IBIS', 'ISBA-CTRIP',  'JULES-ES-1.1', 'OCN', 'ORCHIDEEv3']:
        print("  -Include land fraction: " + files_LSM[0])
        fname_LandFrac     = dir_LSMs + files_LSM[0]
        fname_multiply     = dir_tmp + model + '_NBP_GCB2021_gridarea_corrected.nc'
        os.system('cdo -mul ' + fname_area + ' ' + fname_LandFrac + ' ' + fname_multiply)
        del_multiply = True
    else:
        fname_multiply = fname_area
        del_multiply = False

    #Calculate flux and convert to Pg C
    os.system('cdo -mulc,1e-12 -mul ' + fname_Ysum + ' ' + fname_multiply + ' ' + fname_flux)
    os.system('cdo -z zip_2 setname,SLAND ' + fname_flux + ' ' + fname_SLAND)
    
    #Calculate SLAND density
    os.system('cdo div ' + fname_SLAND + ' ' + fname_area + ' ' + fname_SLden)    
    
    #Loop over different resolutions
    fnames_rename = [fname_SLAND]
    for resolution in resolutions:
        
        #Define output file names
        fname_SLden_reg = dir_tmp + model + '_' + experiment + '_SLANDdensity_NBP_regrid' + resolution + '.nc'
        fname_SLAND_reg = dir_out + model + '_' + experiment + '_SLAND_NBP_regrid' + resolution + '.nc'  
        if os.path.exists(fname_SLAND_reg):  os.remove(fname_SLAND_reg)
        
        #Remap data to common grid
        file_grid = dir_grids + 'grid_xy_' + resolution    

        #Regrid corrected NBP
        os.system('cdo remapcon,' + file_grid + ' ' + fname_SLden + ' ' + fname_SLden_reg)

        #Get cell area of regridded file and calculate regridded SLAND
        os.system('cdo gridarea ' + fname_SLden_reg + ' ' + fname_area_reg)
        os.system('cdo -z zip_2 mul ' + fname_SLden_reg + ' ' + fname_area_reg + ' ' + fname_SLAND_reg)

        #Collect file names for renaming
        fnames_rename.append(fname_SLAND_reg)
        
        #Remove temporary files
        os.remove(fname_area_reg)
        os.remove(fname_SLden_reg)
    
    #Remove temporary files
    os.remove(fname_Ysum_tmp)
    os.remove(fname_Ysum)
    os.remove(fname_area)
    os.remove(fname_flux)
    os.remove(fname_SLden)
    if del_multiply==True:
        os.remove(fname_multiply)
        
    #Rename time_counter to time for ISBA-CTRIP and ORCHIDEEv3
    if model in ['ISBA-CTRIP', 'ORCHIDEEv3']:
        for fname in fnames_rename:
            fname2 = fname[0:-3] + '_tmp.nc'
            data_corr = xr.open_dataset(fname)
            data_corr = data_corr.rename({'time_counter': 'time'})
            encoding = {var: comp for var in data_corr.data_vars}
            data_corr.to_netcdf(fname2, encoding=encoding)
            os.remove(fname)
            os.rename(fname2, fname)
